In [ ]:
cd ../..

In [ ]:
%load_ext autoreload
%autoreload 2

from intertidal.io import export_dataset_metadata, prepare_for_export

In [ ]:
import datacube

dc = datacube.Datacube()
query_params = dict(x=(122.145073, 122.425224), y=(-17.922562, -18.252834))
dd = dc.find_datasets(product="nidem", **query_params)
ds = dc.load(datasets=dd, **query_params).squeeze()
ds["testing"] = ds.nidem
ds["elevation"] = ds.nidem

In [ ]:
ds_processed = prepare_for_export(ds, int_bands=["nidem"], int_nodata=-9999)

dataset_assembler = export_dataset_metadata(
    ds=ds_processed,
    year=2018,
    study_area="x37y30",
    output_location="s3://dea-public-data-dev/derivative/",
    s2_lineage=dd,    
    ls_lineage=dd,
    ancillary_lineage=dd,
    debug=True
)

# export_dataset_metadata(
#     ds=ds,
#     year=2019,
#     region_code="x37y30",
#     output_location="data/processed/",
#     ls_lineage=dd,
#     s2_lineage=dd,
#     ancillary_lineage=dd,
# )

In [6]:
!eo3-validate /home/jovyan/Robbi/dea-intertidal/data/raw/ga_s2ls_intertidal_cyear_3.odc-product.yaml /home/jovyan/Robbi/dea-intertidal/data/processed/ga_s2ls_intertidal_cyear_3/0-0-1a/x094/y145/2023--P1Y/ga_s2ls_intertidal_cyear_3_x094y145_2023--P1Y_final.odc-metadata.yaml --thorough

✓ file:///home/jovyan/Robbi/dea-intertidal/data/raw/ga_s2ls_intertidal_cyear_3.odc-product.yaml
✓ file:///home/jovyan/Robbi/dea-intertidal/data/processed/ga_s2ls_intertidal_cyear_3/0-0-1a/x094/y145/2023--P1Y/ga_s2ls_intertidal_cyear_3_x094y145_2023--P1Y_final.odc-metadata.yaml

valid: 2 paths
